# Time Series Deep Learning Example

This notebook demonstrates a complete workflow for time series forecasting using deep learning models, covering data generation, Exploratory Data Analysis (EDA), data preprocessing, model definition, training, hyperparameter tuning with Optuna, and model saving/loading.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import optuna
import os

# Ensure reproducibility
torch.manual_seed(42)
np.random.seed(42)

## 1. Data Generation (Synthetic Time Series)

We'll create a synthetic time series with a trend, seasonality, and noise for demonstration purposes.

In [ ]:
def generate_time_series(num_points=1000):
    time = np.arange(num_points)
    # Trend
    trend = 0.02 * time
    # Seasonality
    seasonality = 10 * np.sin(time / 20) + 5 * np.cos(time / 50)
    # Noise
    noise = np.random.normal(0, 1, num_points)
    
    data = trend + seasonality + noise
    
    df = pd.DataFrame({'time': time, 'value': data})
    return df

df = generate_time_series()
print(df.head())

## 2. Exploratory Data Analysis (EDA)

Visualize the time series, check for stationarity, and observe patterns.

In [ ]:
plt.figure(figsize=(14, 6))
plt.plot(df['time'], df['value'])
plt.title('Synthetic Time Series')
plt.xlabel('Time')
plt.ylabel('Value')
plt.grid(True)
plt.show()

# Rolling statistics to check for stationarity (visual inspection)
rolling_mean = df['value'].rolling(window=50).mean()
rolling_std = df['value'].rolling(window=50).std()

plt.figure(figsize=(14, 6))
plt.plot(df['time'], df['value'], label='Original')
plt.plot(df['time'], rolling_mean, label='Rolling Mean')
plt.plot(df['time'], rolling_std, label='Rolling Std')
plt.title('Rolling Mean & Standard Deviation')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()

## 3. Data Preprocessing for Deep Learning

We need to transform the time series into sequences (input features) and corresponding target values for supervised learning. This involves creating lagged features and splitting the data into training, validation, and test sets while preserving the temporal order.

In [ ]:
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:(i + seq_length)]
        y = data[i + seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

sequence_length = 50 # Number of past time steps to consider
X, y = create_sequences(df['value'].values, sequence_length)

print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

# Train-Validation-Test Split (Time Series Specific)
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.15)
test_size = len(X) - train_size - val_size

X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size + val_size], y[train_size:train_size + val_size]
X_test, y_test = X[train_size + val_size:], y[train_size + val_size:]

print(f"Train samples: {len(X_train)}")
print(f"Validation samples: {len(X_val)}")
print(f"Test samples: {len(X_test)}")

# Convert to PyTorch tensors
X_train_tensor = torch.from_numpy(X_train).float().unsqueeze(-1) # Add feature dimension
y_train_tensor = torch.from_numpy(y_train).float().unsqueeze(-1)
X_val_tensor = torch.from_numpy(X_val).float().unsqueeze(-1)
y_val_tensor = torch.from_numpy(y_val).float().unsqueeze(-1)
X_test_tensor = torch.from_numpy(X_test).float().unsqueeze(-1)
y_test_tensor = torch.from_numpy(y_test).float().unsqueeze(-1)

# Create DataLoader
batch_size = 64 # This will be tuned by Optuna later
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False) # No shuffle for time series

val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## 4. Deep Learning Model Definition (LSTM)

We'll define a simple LSTM model for time series forecasting.

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :]) # Take the output from the last time step
        return out

## 5. Training and Validation Function

A function to train and evaluate the model, which will be used by Optuna.

In [ ]:
def train_model(model, train_loader, val_loader, optimizer, criterion, n_epochs, trial=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    train_losses = []
    val_losses = []

    for epoch in range(n_epochs):
        model.train()
        running_train_loss = 0.0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_train_loss += loss.item() * inputs.size(0)
        
        epoch_train_loss = running_train_loss / len(train_loader.dataset)
        train_losses.append(epoch_train_loss)

        model.eval()
        running_val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                running_val_loss += loss.item() * inputs.size(0)
        
        epoch_val_loss = running_val_loss / len(val_loader.dataset)
        val_losses.append(epoch_val_loss)

        print(f'Epoch {epoch+1}/{n_epochs}, Train Loss: {epoch_train_loss:.4f}, Val Loss: {epoch_val_loss:.4f}')

        if trial:
            trial.report(epoch_val_loss, epoch)
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()
                
    return model, train_losses, val_losses

## 6. Hyperparameter Tuning with Optuna

We'll use Optuna to find the best hyperparameters for our LSTM model. The objective function will train the model and return the validation loss.

In [ ]:
def objective(trial):
    # Hyperparameters to tune
    hidden_size = trial.suggest_categorical('hidden_size', [32, 64, 128])
    num_layers = trial.suggest_int('num_layers', 1, 3)
    lr = trial.suggest_loguniform('lr', 1e-4, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    n_epochs = trial.suggest_int('n_epochs', 10, 50)

    # Update DataLoader with current batch_size
    train_loader_optuna = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    val_loader_optuna = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    model = LSTMModel(input_size=1, hidden_size=hidden_size, num_layers=num_layers, output_size=1)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    _, _, val_losses = train_model(model, train_loader_optuna, val_loader_optuna, optimizer, criterion, n_epochs, trial)
    
    return val_losses[-1] # Return the final validation loss

In [ ]:
# Create a study and optimize
study_name = "time_series_lstm_optimization"
storage_name = "sqlite:///" + study_name + ".db"

try:
    study = optuna.load_study(study_name=study_name, storage=storage_name)
    print("Loaded existing study.")
except KeyError:
    study = optuna.create_study(direction='minimize', study_name=study_name, storage=storage_name)
    print("Created a new study.")

print("Starting Optuna optimization...")
study.optimize(objective, n_trials=20, timeout=600) # Run 20 trials or for 600 seconds

print("Optimization finished.")
print(f"Number of finished trials: {len(study.trials)}")
print(f"Best trial:")
best_trial = study.best_trial

print(f"  Value (Validation Loss): {best_trial.value:.4f}")
print(f"  Params: ")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

### Optuna Visualization (Requires `plotly` and `matplotlib`)

You can visualize the optimization process and results using Optuna's built-in plotting functions.

In [ ]:
try:
    fig_history = optuna.visualization.plot_optimization_history(study)
    fig_history.show()
    fig_history.write_image("optuna_optimization_history.png")
    print("Optimization history plot saved to optuna_optimization_history.png")
except Exception as e:
    print(f"Could not generate optimization history plot: {e}")

try:
    fig_importance = optuna.visualization.plot_param_importances(study)
    fig_importance.show()
    fig_importance.write_image("optuna_param_importances.png")
    print("Parameter importances plot saved to optuna_param_importances.png")
except Exception as e:
    print(f"Could not generate parameter importances plot: {e}")

try:
    fig_slice = optuna.visualization.plot_slice(study)
    fig_slice.show()
    fig_slice.write_image("optuna_slice_plot.png")
    print("Slice plot saved to optuna_slice_plot.png")
except Exception as e:
    print(f"Could not generate slice plot: {e}")

## 7. Train Final Model with Best Hyperparameters

After finding the best hyperparameters, we train the model on the combined training and validation data (or just training data, depending on strategy) and evaluate on the test set.

In [ ]:
best_params = best_trial.params

final_model = LSTMModel(input_size=1,
                        hidden_size=best_params['hidden_size'],
                        num_layers=best_params['num_layers'],
                        output_size=1)

final_optimizer = optim.Adam(final_model.parameters(), lr=best_params['lr'])
final_criterion = nn.MSELoss()

# Combine train and validation data for final training
X_train_val_tensor = torch.cat((X_train_tensor, X_val_tensor), dim=0)
y_train_val_tensor = torch.cat((y_train_tensor, y_val_tensor), dim=0)

train_val_dataset = TensorDataset(X_train_val_tensor, y_train_val_tensor)
train_val_loader = DataLoader(train_val_dataset, batch_size=best_params['batch_size'], shuffle=False)

print("Training final model with best hyperparameters...")
final_model, final_train_losses, final_val_losses = train_model(final_model, train_val_loader, val_loader, final_optimizer, final_criterion, best_params['n_epochs'])

plt.figure(figsize=(12, 5))
plt.plot(final_train_losses, label='Final Train Loss')
plt.plot(final_val_losses, label='Final Validation Loss')
plt.title('Final Model Training & Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

## 8. Model Evaluation on Test Set

Evaluate the trained model on the unseen test data.

In [ ]:
final_model.eval()
test_loss = 0.0
predictions = []
true_values = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
final_model.to(device)

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = final_model(inputs)
        loss = final_criterion(outputs, targets)
        test_loss += loss.item() * inputs.size(0)
        predictions.extend(outputs.cpu().numpy().flatten())
        true_values.extend(targets.cpu().numpy().flatten())

test_loss /= len(test_loader.dataset)
print(f'Test Loss: {test_loss:.4f}')

plt.figure(figsize=(14, 6))
plt.plot(true_values, label='True Values')
plt.plot(predictions, label='Predictions')
plt.title('Test Set Predictions vs True Values')
plt.xlabel('Time Step')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()

## 9. Model Saving and Loading

Save the trained model's state dictionary and demonstrate how to load it.

In [ ]:
model_save_path = 'best_time_series_model.pth'
torch.save(final_model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

# Demonstrate loading the model
loaded_model = LSTMModel(input_size=1,
                         hidden_size=best_params['hidden_size'],
                         num_layers=best_params['num_layers'],
                         output_size=1)
loaded_model.load_state_dict(torch.load(model_save_path))
loaded_model.eval()
print("Model loaded successfully!")

# You can now use loaded_model for inference
# Example: Make a prediction with the loaded model
sample_input = X_test_tensor[0:1].to(device)
loaded_prediction = loaded_model(sample_input).item()
print(f"Sample input: {sample_input.cpu().numpy().flatten()}")
print(f"Prediction from loaded model for sample input: {loaded_prediction:.4f}")
print(f"True value for sample input: {y_test_tensor[0].item():.4f}")

## 10. Further Considerations

-   **Feature Scaling**: For real-world data, it's crucial to scale your time series data (e.g., using `MinMaxScaler` or `StandardScaler`) before feeding it to neural networks.
-   **More Complex Models**: Explore more advanced architectures like GRUs, Transformers, or even CNNs for time series.
-   **Multivariate Time Series**: Extend the model to handle multiple input features.
-   **Forecasting Horizon**: Adapt the model to predict multiple future time steps.
-   **Error Metrics**: Use appropriate time series error metrics like MAE, RMSE, MAPE.
-   **Cross-Validation**: For more robust evaluation, consider time series cross-validation strategies (e.g., rolling origin cross-validation).